In [2]:
# https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed
# 1. Load document
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./mh_docs/masterigrandview.pdf")

#Load the document by calling loader.load()
pages = loader.load()

print(len(pages))
print(pages[0].page_content[0:500])

print(pages[0].metadata)
# {'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}

63
Chào đón
SIÊU PHẨM CAO TẦNG ĐẦU TIÊN
tại The Global City

{'source': './mh_docs/masterigrandview.pdf', 'page': 0}


In [3]:
# 2. Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(pages)
len(docs)

62

In [4]:
# 3. Vector store
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import getpass
import os

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

os.environ["LANGSMITH_TRACING_V2"] = "true"
if not os.getenv("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

persist_directory = './docs/chroma/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

248


In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [6]:
# Query vector db
question = "Dự án có bao nhiêu tiện ích ?"

docs = vectordb.similarity_search(question,k=3)

# Check the length of the document
len(docs)

for doc in docs:
    print(doc.metadata)

print(docs[0].page_content)

{'page': 50, 'source': './mh_docs/masterigrandview.pdf'}
{'page': 50, 'source': './mh_docs/masterigrandview.pdf'}
{'page': 50, 'source': './mh_docs/masterigrandview.pdf'}
HỆ TIỆN ÍCH ẤN TƯỢNG
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
GHẾ NẰM THƯ 
GIÃN DƯỚI NƯỚC
HỒ BƠI VÔ CỰC
GHẾ THƯ GIÃN 
TẮM NẮNG
HỒ BƠI TRẺ EM
SEN TẮM ĐỨNG
NGOÀI TRỜI
VƯỜN ĐỌC SÁCH
VƯỜN YOGA/THIỀN
KHU TẬP THỂ THAO 
NGOÀI TRỜI
KHUÔN VIÊN BBQ
SÂN CHƠI TRẺ 
EM
KHU VUI CHƠI TRẺ 
EM TRONG NHÀ
PHÒNG CHƠI GAME
VƯỜN THƯ 
GIÃN
PHÒNG ĐA 
CHỨC NĂNG
KHU VỰC 
BUSINESS CENTER
THƯ VIỆN
LOUNGE 
PHÒNG THAY ĐỒ
PHÒNG GYM 
TRONG NHÀ
PHÒNG YOGA


In [7]:
# Retrieval opt2
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """Bạn là nhân viên chăm sóc khách hàng của Masterise Homes và bạn sẽ trả lời các câu hỏi của khách hàng về Công ty cũng như các dự án thuộc Công ty. 
      Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em. 
      Hãy trả lời câu hỏi của khách hàng một cách lễ phép, đầy đủ và tôn trọng. Sử dụng các từ như 'Dạ thưa', 'Xin chào quý khách', 'Em xin phép giải thích', 'Rất cảm ơn quý khách đã hỏi',... để thể hiện thái độ lịch sự và chuyên nghiệp.. 
      Trong trường hợp khách hàng hỏi những câu hỏi không liên quan đến Công ty và dự án, hãy từ chối trả lời một cách lịch sự.
      Nếu như trong lần trả lời này bạn không còn thông tin gì thêm ngoài các thông tin đã trả lời ở những lần trước đó thì hãy dừng công cụ và phản hồi khách hàng là bạn đã trả lời tất cả các thông tin bạn biết. Nếu khách hàng muốn tìm hiểu thêm thì liên hệ với số hotline của dự án. Không cần xin lỗi khách hàng khi bạn thiếu thông tin.
      

{context}

Question: {question}

Helpful Answer:"""

# Define prompt for question-answering
prompt = PromptTemplate.from_template(template)


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vectordb.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [8]:
response = await graph.ainvoke({"question": "Tại sao tôi nên mua Masteri Grand View?"})
print(response)

{'question': 'Tại sao tôi nên mua Masteri Grand View?', 'context': [Document(metadata={'page': 61, 'source': './mh_docs/masterigrandview.pdf'}, page_content='NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTIÊN \nTẠI TRUNG TÂM MỚI\nVỊ TRÍ KIM CƯƠNG TẠI GIAO \nĐIỂM ĐA SẮC MÀU\nĐƯỢC PHÁT TRIỂN BỞI CÁC \nTÊN TUỔI HÀNG ĐẦU THẾ GIỚI\nTHỜI ĐIỂM VÀNG ĐỂ SỞ HỮU \nNHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE \nGLOBAL CITY\nTHỪA HƯỞNG TOÀN BỘ HỆ TIỆN \nÍCH ĐỈNH CAO CỦA \nTHE GLOBAL CITY\n7.\n8.\nSỞ HỮU LOẠI HÌNH 4PN CỰC KHAN \nHIẾM TRÊN THỊ TRƯỜNG\nMỖI SÀN CHỈ 8CĂN'), Document(metadata={'page': 61, 'source': './mh_docs/masterigrandview.pdf'}, page_content='NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW \nTRỞ THÀNH LỰA CHỌN HÀNG ĐẦU\n1.\n2.\n3.\n4.\n5.\n6.\nTỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –\nTHE GLOBAL CITY –\nQUỸ ĐẤT VÀNG TẠI P. AN PHÚ\nLÀ SIÊU PHẨM CAO TẦNG ĐẦU \nTI

In [9]:
# Memory
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vectordb.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [10]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "Bạn là nhân viên chăm sóc khách hàng của Masterise Homes và bạn sẽ trả lời các câu hỏi của khách hàng về Công ty cũng như các dự án thuộc Công ty."
        "Hãy dùng đại từ xưng hô gọi khách hàng là Anh/Chị, còn bạn dùng đại từ xưng hô là Em."
        "Hãy trả lời câu hỏi của khách hàng một cách lịch sự và tôn trọng."
        "Trong trường hợp khách hàng hỏi những câu hỏi không liên quan đến Công ty và dự án, hãy từ chối trả lời một cách lịch sự."
        "Không bao giờ được hỏi khách hỏi xong chưa và xin phép dừng cuộc hội thoại"
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [11]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [12]:
input_message = "Giới thiệu về dự án Masteri Grand View"
config = {"configurable": {"thread_id": "abc123"}}

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Giới thiệu về dự án Masteri Grand View
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_WMyQ6qxg7eZrgENY223adgCJ)
 Call ID: call_WMyQ6qxg7eZrgENY223adgCJ
  Args:
    query: dự án Masteri Grand View
================================= Tool Message =================================
Name: retrieve

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 
NHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE 
GLOBAL CITY
THỪA HƯỞNG TOÀN BỘ HỆ TIỆN 
ÍCH ĐỈNH CAO CỦA 
THE GLOBAL CITY
7.
8.
SỞ HỮU LOẠI HÌNH 4PN CỰC KHAN 
HIẾM TRÊN 

In [13]:
input_message = "Còn gì nữa không ?"
config = {"configurable": {"thread_id": "abc123"}}

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Còn gì nữa không ?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_jSD3SeRtWgFhryZ2Qf4YAFb9)
 Call ID: call_jSD3SeRtWgFhryZ2Qf4YAFb9
  Args:
    query: Masteri Grand View thông tin chi tiết
================================= Tool Message =================================
Name: retrieve

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 
NHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE 
GLOBAL CITY
THỪA HƯỞNG TOÀN BỘ HỆ TIỆN 
ÍCH ĐỈNH CAO CỦA 
THE GLOBAL CITY
7.
8.
SỞ HỮU LOẠI HÌNH 4PN CỰC KHAN 
HIẾM TRÊN THỊ TRƯ

In [14]:
input_message = "Còn gì nữa không ?"
config = {"configurable": {"thread_id": "abc123"}}

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Còn gì nữa không ?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_1h2aLxl823omRDDIf7UGmygG)
 Call ID: call_1h2aLxl823omRDDIf7UGmygG
  Args:
    query: Masteri Grand View thông tin bổ sung
================================= Tool Message =================================
Name: retrieve

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 
NHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE 
GLOBAL CITY
THỪA HƯỞNG TOÀN BỘ HỆ TIỆN 
ÍCH ĐỈNH CAO CỦA 
THE GLOBAL CITY
7.
8.
SỞ HỮU LOẠI HÌNH 4PN CỰC KHAN 
HIẾM TRÊN THỊ TRƯỜ

In [15]:
input_message = "Còn gì nữa không ?"
config = {"configurable": {"thread_id": "abc123"}}

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Còn gì nữa không ?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_mW7Co8j9weGvi1exU5ceY55E)
 Call ID: call_mW7Co8j9weGvi1exU5ceY55E
  Args:
    query: Masteri Grand View thông tin mới nhất
================================= Tool Message =================================
Name: retrieve

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 
NHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE 
GLOBAL CITY
THỪA HƯỞNG TOÀN BỘ HỆ TIỆN 
ÍCH ĐỈNH CAO CỦA 
THE GLOBAL CITY
7.
8.
SỞ HỮU LOẠI HÌNH 4PN CỰC KHAN 
HIẾM TRÊN THỊ TRƯ

In [17]:
input_message = "Cho anh thông tin tiện ích"
config = {"configurable": {"thread_id": "abc123"}}

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Cho anh thông tin tiện ích
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_dpmd3JQnPzKihRNpnUKiZWUr)
 Call ID: call_dpmd3JQnPzKihRNpnUKiZWUr
  Args:
    query: tiện ích Masteri Grand View
================================= Tool Message =================================
Name: retrieve

Source: {'page': 61, 'source': './mh_docs/masterigrandview.pdf'}
Content: NHỮNG LÝ DO KHIẾN MASTERI GRAND VIEW 
TRỞ THÀNH LỰA CHỌN HÀNG ĐẦU
1.
2.
3.
4.
5.
6.
TỌA LẠC TRONG KHU ĐÔ THỊ 117.4HA –
THE GLOBAL CITY –
QUỸ ĐẤT VÀNG TẠI P. AN PHÚ
LÀ SIÊU PHẨM CAO TẦNG ĐẦU 
TIÊN 
TẠI TRUNG TÂM MỚI
VỊ TRÍ KIM CƯƠNG TẠI GIAO 
ĐIỂM ĐA SẮC MÀU
ĐƯỢC PHÁT TRIỂN BỞI CÁC 
TÊN TUỔI HÀNG ĐẦU THẾ GIỚI
THỜI ĐIỂM VÀNG ĐỂ SỞ HỮU 
NHỮNG CĂN HỘ ĐẦU TIÊN TẠI THE 
GLOBAL CITY
THỪA HƯỞNG TOÀN BỘ HỆ TIỆN 
ÍCH ĐỈNH CAO CỦA 
THE GLOBAL CITY
7.
8.
SỞ HỮU LOẠI HÌNH 4PN CỰC KHAN 
HIẾM TRÊN THỊ TRƯỜN